<a href="https://colab.research.google.com/github/Mohamed-derbeli/Fuel-cell-system/blob/main/Modeling_of_a_Commercial_Heliocentris_PEMFC_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

In [ ]:
with open("./training_cleaned.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        list_item=[]    
        ### START CODE HERE
        #Create list items where the first item is the text, found in row[5], and the second is the label. Note that the label is a '0' or a '4' in the text. When it's the former, make
        # your label to be 0, otherwise 1. Keep a count of the number of sentences in num_sentences
        list_item.append(row[5])
        if row[0]=='0':
          list_item.append(0)
        else:
            list_item.append(1)
        ### END CODE HERE
        num_sentences = num_sentences + 1
        corpus.append(list_item)

In [ ]:
mydata = pd.read_excel("https://www.eia.gov/dnav/pet/hist_xls/RBRTEd.xls",sheetname="Data 1", skiprows=2)
If you do not specify name of sheet in sheetname= option, it would take by default first sheet.

In [ ]:
#Inputs/ F: Fan power, T: Temperature, I_stack: Stack current, 
#Outputs/ V_stack: Stack voltage

F= 


